<h1>Découvrez comment créer un chatbot RAG local en utilisant DeepSeek-R1 avec Ollama, LangChain et Chroma.</h1>

**Pourquoi utiliser DeepSeek-R1 avec RAG ?**
 <p>DeepSeek-R1 est idéal pour les systèmes basés sur RAG en raison de ses performances optimisées, de ses capacités avancées de recherche vectorielle et de sa flexibilité dans différents environnements, des configurations locales aux déploiements évolutifs. Voici quelques raisons pour lesquelles il est efficace :</p>

1. **Récupération haute performance :** DeepSeek-R1 gère de grandes
collections de documents avec une faible latence.
2. **Classement de pertinence précis :** il garantit une récupération précise des passages en calculant la similarité sémantique.
3. **Avantages en termes de coût et de confidentialité :** vous pouvez exécuter DeepSeek-R1 localement pour éviter les frais d’API et sécuriser les données sensibles.
4. **Intégration facile :** Il s'intègre facilement aux bases de données vectorielles comme Chroma .
5. **Capacités hors ligne :** avec DeepSeek-R1, vous pouvez créer des systèmes de récupération qui fonctionnent même sans accès Internet une fois le modèle téléchargé.

**Processus :**
Le processus commence par le chargement et le fractionnement d'un PDF en blocs de texte, suivi de la génération d'incorporations pour ces blocs. Ces incorporations sont stockées dans une base de données Chroma pour une récupération efficace. Lorsqu'un utilisateur soumet une requête, le système récupère les blocs de texte les plus pertinents et utilise DeepSeek-R1 pour générer une réponse en fonction du contexte récupéré.

# Étape 1 : Prérequis


In [ ]:
!python --version

Python 3.11.11


In [ ]:
#!pip install langchain chromadb gradio ollama
#!pip install -U langchain-community

In [ ]:
#!pip install pymupdf

In [ ]:
import ollama
import re
import gradio as gr
from concurrent.futures import ThreadPoolExecutor
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from chromadb.config import Settings
from chromadb import Client
from langchain.vectorstores import Chroma

# Étape 2 : charger le PDF à l'aide de PyMuPDFLoader

Nous utiliserons LangChain PyMuPDFLoaderpour extraire le texte de la version PDF du livre Foundations of LLMs de Tong Xiao et Jingbo Zhu. Il s'agit d'un livre très axé sur les mathématiques, ce qui signifie que notre chatbot devrait être capable d'expliquer correctement les mathématiques qui sous-tendent les LLM. Vous pouvez trouver le livre sur [arXiv ](https://arxiv.org/abs/2501.09223).

In [ ]:
!ls /content/

'Foundations of Large Language Models.pdf'   sample_data


In [ ]:
# Load the document using PyMuPDFLoader
loader = PyMuPDFLoader("/content/Foundations of Large Language Models.pdf")

documents = loader.load()

In [ ]:
assert len(documents) > 0, "Erreur : Aucun document chargé !"
print("✅ Chargement du PDF réussi :", len(documents), "documents extraits.")

✅ Chargement du PDF réussi : 231 documents extraits.


# Étape 3 : diviser le document en morceaux plus petits

Nous allons diviser le texte extrait en morceaux plus petits et superposés pour une meilleure récupération du contexte. Vous pouvez faire varier la taille du morceau et le chevauchement des morceaux en fonction de votre système dans la **RecursiveCharacterTextSpilitter()** fonction.

In [ ]:
# Split the document into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunks = text_splitter.split_documents(documents)

In [ ]:
assert len(chunks) > 0, "Erreur : Aucun chunk généré !"
print("✅ Découpage réussi :", len(chunks), "chunks créés.")

✅ Découpage réussi : 821 chunks créés.


# Étape 4 : générer des intégrations à l'aide de DeepSeek-R1

<p>Nous utiliserons Ollama Embeddings basé sur DeepSeek-R1 pour générer les intégrations de documents. Selon la taille du document, la génération d'intégrations peut prendre du temps, il est donc préférable de la paralléliser pour un traitement plus rapide.</p>

<p> **Remarque :** model="deepseek-r1" par défaut, le modèle de paramètre 7B est pris en compte. Vous pouvez le modifier selon vos besoins en 8B, 14B, 32B, 70B ou 671B. Remplacez X dans le nom du modèle suivant par la taille du modèle :model="deepseek-r1:X"</p>

In [ ]:
#!pip install sentence-transformers

from langchain.embeddings import HuggingFaceEmbeddings

embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

<ipython-input-7-a69c0daa07d4>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mode

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Parallelize embedding generation
def generate_embedding(chunk):
    return embedding_function.embed_query(chunk.page_content)


with ThreadPoolExecutor() as executor:
    embeddings = list(executor.map(generate_embedding, chunks))

In [ ]:
assert len(embeddings) == len(chunks), "Erreur : Le nombre d'embeddings ne correspond pas aux chunks !"
print("✅ Génération des embeddings réussie :", len(embeddings), "embeddings générés.")

✅ Génération des embeddings réussie : 821 embeddings générés.


# Étape 5 : Stocker les intégrations dans Chroma Vector Store

Nous stockerons les intégrations et les blocs de texte correspondants dans une base de données vectorielle hautes performances, Chroma.

In [ ]:
# Initialize Chroma client and create/reset the collection
client = Client(Settings())
#client.delete_collection(name="foundations_of_llms")  # Delete existing collection (if any)
collection = client.create_collection(name="foundations_of_llms")

# Add documents and embeddings to Chroma
for idx, chunk in enumerate(chunks):
    collection.add(
        documents=[chunk.page_content],
        metadatas=[{'id': idx}],
        embeddings=[embeddings[idx]],
        ids=[str(idx)]  # Ensure IDs are strings
    )

In [ ]:
assert collection.count() == len(chunks), f"Erreur : {collection.count()} documents stockés, attendu {len(chunks)}"
print("✅ Stockage des embeddings réussi dans Chroma.")

✅ Stockage des embeddings réussi dans Chroma.


# Étape 6 : Initialiser le Retriever


Nous allons initialiser le récupérateur Chroma, en veillant à ce qu'il utilise les mêmes intégrations DeepSeek-R1 pour les requêtes.

In [ ]:
# Initialize retriever using Ollama embeddings for queries
retriever = Chroma(collection_name="foundations_of_llms", client=client, embedding_function=embedding_function).as_retriever()

<ipython-input-12-220fe67ecda8>:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  retriever = Chroma(collection_name="foundations_of_llms", client=client, embedding_function=embedding_function).as_retriever()


In [ ]:
test_question = "What are Large Language Models?"
retrieved_docs = retriever.invoke(test_question)

assert len(retrieved_docs) > 0, "Erreur : Aucun document récupéré !"
print("✅ Retriever fonctionne correctement :", len(retrieved_docs), "documents récupérés.")

✅ Retriever fonctionne correctement : 4 documents récupérés.


In [ ]:
for doc in retrieved_docs:
  print(doc)
  print(50*'-')

page_content='arXiv:2501.09223v1  [cs.CL]  16 Jan 2025
Foundations of
Large Language Models
Tong Xiao and Jingbo Zhu
January 17, 2025
NLP Lab, Northeastern University & NiuTrans Research' metadata={'id': 0}
--------------------------------------------------
page_content='Preface
Large language models originated from natural language processing, but they have undoubtedly
become one of the most revolutionary technological advancements in the ﬁeld of artiﬁcial intelli-
gence in recent years. An important insight brought by large language models is that knowledge
of the world and languages can be acquired through large-scale language modeling tasks, and
in this way, we can create a universal model that handles diverse problems. This discovery has
profoundly impacted the research methodologies in natural language processing and many related
disciplines. We have shifted from training specialized systems from scratch using a large amount
of labeled data to a new paradigm of using large-scale 

Le récupérateur Chroma se connecte à la collection « foundations_of_llms » et utilise les intégrations DeepSeek-R1 via Ollama pour intégrer les requêtes des utilisateurs. Il récupère les fragments de documents les plus pertinents en fonction de la similarité des vecteurs pour les réponses contextuelles.

# Étape 7 : Définir le pipeline RAG

Ensuite, nous récupérerons les morceaux de texte les plus pertinents et les formaterons pour que DeepSeek-R1 génère des réponses.


def retrieve_context(question):

In [ ]:
def retrieve_context(question):
    # Retrieve relevant documents
    results = retriever.invoke(question)

    # Combine the retrieved content
    context = "\n\n".join([doc.page_content for doc in results])
    return context

In [ ]:
context = retrieve_context(test_question)

assert len(context) > 0, "Erreur : Contexte vide !"
print("✅ Contexte récupéré :", len(context.split()), "mots extraits.")

✅ Contexte récupéré : 431 mots extraits.


La retrieve_context fonction intègre la requête utilisateur à l'aide de DeepSeek-R1 et récupère les principaux fragments de documents pertinents via le récupérateur Chroma. Elle combine ensuite le contenu des fragments récupérés dans une seule chaîne de contexte pour un traitement ultérieur.

# Étape 8 : interrogez DeepSeek-R1 pour obtenir des réponses contextuelles

In [ ]:
!pip install -q transformers accelerate bitsandbytes
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.4 MB/s eta 0:00:00


In [ ]:
!pip install -U bitsandbytes

In [ ]:
from huggingface_hub import login

login()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Chargement en 4-bit
    bnb_4bit_compute_dtype="float16",  # Utilisation de FP16
    bnb_4bit_use_double_quant=True  # Double quantization pour optimiser
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto"
)

ERROR:bitsandbytes.cextension:Could not load bitsandbytes native library: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.32' not found (required by /usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbytes_cpu.so)
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 85, in <module>
    lib = get_native_library()
          ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 72, in get_native_library
    dll = ct.cdll.LoadLibrary(str(binary_path))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 454, in LoadLibrary
    return self._dlltype(name)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4

RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

Voici un exemple de texte que tu peux tester avec ton modèle Mistral-7B-Instruct-v0.2 après l’avoir chargé en 4-bit :

In [ ]:
from transformers import pipeline

tokenizer.pad_token = tokenizer.eos_token

# Création du pipeline de génération
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [ ]:
# Exemple de texte à compléter
prompt = "Explique-moi la théorie de la relativité en termes simples."

# Générer du texte avec le modèle
#output = pipe(prompt, max_length=150, do_sample=True, temperature=0.7)
output = pipe(prompt, max_length=150, do_sample=True, temperature=0.7, truncation=True)


{'generated_text': 'Explique-moi la théorie de la relativité en termes simples. La théorie de la relativité est une théorie physique développée par Albert Einstein dans les années 1900. Il y a deux types de relativité : la relativité spéciale et la relativité générale.\n\nLa relativité spéciale (SR) décrit les lois de la physique pour des objets en mouvement uniforme, à des vitesses inférieures à celle de la lumière. Elle établit que:\n\n1. La vitesse de la lumière dans un vide est constante, indépendante de la vitesse de l'}


In [ ]:
# Afficher la réponse
print(output[0]['generated_text'])

Explique-moi la théorie de la relativité en termes simples. La théorie de la relativité est une théorie physique développée par Albert Einstein dans les années 1900. Il y a deux types de relativité : la relativité spéciale et la relativité générale.

La relativité spéciale (SR) décrit les lois de la physique pour des objets en mouvement uniforme, à des vitesses inférieures à celle de la lumière. Elle établit que:

1. La vitesse de la lumière dans un vide est constante, indépendante de la vitesse de l


In [ ]:
'''def query_deepseek(question, context):
    # Format the input prompt
    formatted_prompt = f"Question: {question}\n\nContext: {context}"

    # Query DeepSeek-R1 using Ollama
    response = embedding_function.chat(
        model="deepseek-r1",
        messages=[{'role': 'user', 'content': formatted_prompt}]
    )

    # Clean and return the response
    response_content = response['message']['content']
    final_answer = re.sub(r'<think>.*?</think>', '', response_content, flags=re.DOTALL).strip()
    return final_answer'''

In [ ]:
def query_mistral(question, context):
    # Format the input prompt
    formatted_prompt = f"Question: {question}\n\nContext: {context}"

    # Query Mistral
    output = pipe(prompt, max_length=150, do_sample=True, temperature=0.7, truncation=True)

    # Clean and return the response
    response_content = output[0]['generated_text']
    return response_content

In [ ]:
test_question = "What are Large Language Models?"
context = retrieve_context(test_question)

test_answer = query_mistral(test_question, context)  # Utilisation de Mistral au lieu de DeepSeek

assert len(test_answer) > 0, "Erreur : Réponse vide !"
print("✅ Mistral-7B-Instruct a généré une réponse :", test_answer[:], "...")

✅ Mistral-7B-Instruct a généré une réponse : Explique-moi la théorie de la relativité en termes simples. La théorie de la relativité est une théorie physique développée par Albert Einstein dans les années 1900. Il y a deux types de relativité : la relativité spéciale et la relativité générale.

La relativité spéciale (SR) décrit les lois de la physique pour des objets en mouvement uniforme, à des vitesses inférieures à celle de la lumière. Elle établit que:

1. La vitesse de la lumière dans un vide est constante, indépendante de la vitesse de l ...


Pour obtenir la réponse finale, nous commençons par combiner la question de l'utilisateur et le contexte récupéré dans une invite structurée. Ensuite, nous envoyons cette invite au modèle DeepSeek-R1 via Ollama pour recevoir une réponse. Pour rendre le résultat final présentable, nous supprimons les balises inutiles et renvoyons la réponse finale.

# Etape 9 : Poser des question

In [ ]:
def ask_question(question):
    # Retrieve context and generate an answer using RAG
    context = retrieve_context(question)
    answer = query_mistral(question, context)
    return answer

# Étape 9 : Créer l'interface Gradio


In [ ]:
print(ask_question('what is the main components of LLMs ?'))

APIStatusError: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly allowance.'}

Notre pipeline RAG est en place. Nous allons maintenant utiliser Gradio pour créer une interface interactive permettant aux utilisateurs de poser des questions liées à sa base de connaissances (Fondements des LLM dans ce cas).

In [ ]:
# Set up the Gradio interface
interface = gr.Interface(
    fn=ask_question,
    inputs="text",
    outputs="text",
    title="RAG Chatbot: Foundations of LLMs",
    description="Ask any question about the Foundations of LLMs book. Powered by DeepSeek-R1."
)

In [ ]:
interface.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5196409dfcfecad7c2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5ca2443e4bca0a210f.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://9b39ef29c21d957685.gradio.live
Killing tunnel 127.0.0.1:7862 <> https://5196409dfcfecad7c2.gradio.live
